# INITIAL IMPLEMENTATION OF WORDGRAPH2VEC ALGORITHM

WordGraph2Vec algorithm in skip-gram model (TF Implementation)

Most skip-gram W2V + embedding lookup table and negative sampling are included in tensorflow:
```
tf.nn.embedding_lookup
tf.nn.sampled_softmax_loss
```

Steps:

1.   <b>Preprocessing</b><br>
Tokenize input and convert input into INT representation. Have look ups: <br> word $→$ int index
2.   <b>Subsampling</b> <br>
Words (e.g stopwords) does not provide much context of words nearby. Discard them to remove some of the noise from our data, get faster training and better representation. i.e Subsampling by Mikolov (Discard word in training set, discard it with probability: p(word) = 1 - sqrt(threshold/freq(word))
3.   **Sampling Table**:
```
inputs and labels
```
**labels** = 2dim required by `tf.nn.sampled_softmax_loss` used for negative sampling.<br>
**embedding matrix** = [number_of_words x number_of_units(hidden layer)]
$→$ if 10000 words and 300 hidden units matrix will have size $10000\times 300$
**tokenized words** = input ~⏯ tokenized data of inputs (integers = # of words in vocabulary)
4.   **Negative Sampling**<br>
Update the weights for correct label, only a small number of incorrect labels (negative sampling) `tf.nn.sampled_softmax_loss`
5.   Visualization





One hot encoding thousands of classes to predict each word is massively inefficient. Matrix multiplication going into first hidden layer will almost all of resulting values be zero.(Sparse)

Idea: Get embeddings weights from an embedding layer. One-hot encoded vector with a matrix return row of matrix corresponding the index of the on input unit. Weight Matrix = lookup table, encode the words as integers and then get the index $i^{th}$ = embedding lookup and the # of hidden units is embedding dimension.

Embedding lookup is weight matrix.

Embedding layer is a hidden layer. Lookup is a shortcut for matrix. Embedding layer is just a hidden layer. (Embeddings used only for Words that contain semantic meaning)


## Word2Vec

Finds efficient representations (vectors that represent the words) = Vectors also contain semantic information about the words. <br>
e.g black whte red will have vectors near each other.


In [ ]:
import time
import numpy as np
# import tensorflow as tf
import tensorflow.compat.v1 as tf
from urllib.request import urlretrieve
from os.path import isfile, isdir
from tqdm import tqdm
import zipfile

In [ ]:
dataset_folder_path = 'content'
dataset_filename = 'text8.zip'
dataset_name = 'Text8 Dataset'

class DLProgress(tqdm):
    last_block = 0

    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num

if not isfile(dataset_filename):
    with DLProgress(unit='B', unit_scale=True, miniters=1, desc=dataset_name) as pbar:
        urlretrieve(
            'http://mattmahoney.net/dc/text8.zip',
            dataset_filename,
            pbar.hook)

if not isdir(dataset_folder_path):
    with zipfile.ZipFile(dataset_filename) as zip_ref:
        zip_ref.extractall(dataset_folder_path)

with open('content/text8') as f:
    text = f.read()

### Preprocessing

- Convert any punctuation into tokens.
- Remove words that show up 5 times or less in dataset (Reduce noise and improve quality of the vector)

In [ ]:
import re
from collections import Counter
def preprocess(text):
    # Replace punctuation with tokens so we can use them in our model
    text = text.lower()
    text = text.replace('.', ' <PERIOD> ')
    text = text.replace(',', ' <COMMA> ')
    text = text.replace('"', ' <QUOTATION_MARK> ')
    text = text.replace(';', ' <SEMICOLON> ')
    text = text.replace('!', ' <EXCLAMATION_MARK> ')
    text = text.replace('?', ' <QUESTION_MARK> ')
    text = text.replace('(', ' <LEFT_PAREN> ')
    text = text.replace(')', ' <RIGHT_PAREN> ')
    text = text.replace('--', ' <HYPHENS> ')
    text = text.replace('?', ' <QUESTION_MARK> ')
    # text = text.replace('\n', ' <NEW_LINE> ')
    text = text.replace(':', ' <COLON> ')
    words = text.split()
    # Remove all words with  5 or fewer occurences
    word_counts = Counter(words)
    trimmed_words = [word for word in words if word_counts[word] > 5]
    return trimmed_words


def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: A list where each item is a tuple of (batch of input, batch of target).
    """
    n_batches = int(len(int_text) / (batch_size * seq_length))
    # Drop the last few characters to make only full batches
    xdata = np.array(int_text[: n_batches * batch_size * seq_length])
    ydata = np.array(int_text[1: n_batches * batch_size * seq_length + 1])
    x_batches = np.split(xdata.reshape(batch_size, -1), n_batches, 1)
    y_batches = np.split(ydata.reshape(batch_size, -1), n_batches, 1)
    return list(zip(x_batches, y_batches))


def create_lookup_tables(words):
    """
    Create lookup tables for vocabulary
    :param words: Input list of words
    :return: A tuple of dicts.  The first dict....
    """
    word_counts = Counter(words)
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    int_to_vocab = {ii: word for ii, word in enumerate(sorted_vocab)}
    vocab_to_int = {word: ii for ii, word in int_to_vocab.items()}
    return vocab_to_int, int_to_vocab

In [ ]:
words = preprocess(text)
print(words[:30])

['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english', 'revolution', 'and', 'the', 'sans', 'culottes', 'of', 'the', 'french', 'revolution', 'whilst']


In [ ]:
print("Total words: {}".format(len(words)))
print("Unique words: {}".format(len(set(words))))

Total words: 16680599
Unique words: 63641


In [ ]:
vocab_to_int, int_to_vocab = create_lookup_tables(words)
int_words = [vocab_to_int[word] for word in words]

In [ ]:
int_to_vocab[int_words[0]]

'anarchism'

### Subsampling

Discard some of the stopwords as they do not provide a lot of context for nearby words. $⇒$ Faster training, better representation and less noise.

$⇒$ Subsampling

For each word $w_i$ in dataset discard it with probability of:
$$
P(w_i) = 1 - \sqrt(\frac{t}{f(w_i})
$$

In [ ]:
from collections import Counter
import random

threshold = 1e-5
word_counts = Counter(int_words)
total_count = len(int_words)
freqs = {word: count/total_count for word, count in word_counts.items()}
p_drop = {word: 1 - np.sqrt(threshold/freqs[word]) for word in word_counts}
train_words = [word for word in int_words if random.random() < (1 - p_drop[word])]

### Making data for training (Batches of words)

Skip-Gram: Each word in text grap all words in a window around that word (Size C) Distant words are less related to the current words close to it (less weight to distant words by sampling less from those words in our training example)

If `win_size = 5` for each training word we select randomly a number R in range [1, C] and then use R words from history and R words from future of current word as correct label = 1

In [ ]:
def get_target(words, idx, win_size=5):
    ''' Get a list of words in a window around an index '''
    R = np.random.randint(1, win_size + 1)
    start = idx - R if (idx - R) > 0 else 0
    stop = idx + R
    target_words = set(words[start:idx] + words[idx + 1: stop + 1])
    return list(target_words)

In [ ]:
def get_batches(words, batch_size, win_size=5):
    '''
    Create a generator of word batches as a tuple (inputs, targets)
    '''
    n_batches = len(words)//batch_size
    words = words[:n_batches * batch_size]
    for idx in range(0, len(words), batch_size):
        x, y = [], []
        batch = words[idx:idx+batch_size]
        for ii in range(len(batch)):
            batch_x = batch[ii]
            batch_y = get_target(batch, ii, win_size)
            y.extend(batch_y)
            x.extend([batch_x] * len(batch_y))
        yield x, y

In [ ]:
train_graph = tf.Graph()
with train_graph.as_default():
    inputs = tf.compat.v1.placeholder(tf.int32, [None], name='inputs')
    labels = tf.compat.v1.placeholder(tf.int32, [None, None], name='labels')

### Embeddings

Size : [# of words x # of hidden units]<br>
Tokenized data as input (# of tokens == # of words)


`tf.nn.embedding_lookup` = pass in embedding matrix and a tensor of integers and it returns rows in the matrix corresponding to those integers.

<br>
# of embeddings = 200
<br>
Create embeddin matrix variable and use tf.nn.embedding_lookup to get embedding tensors. For embedding matrix (initialize it with a uniform random number between -1 and 1 using `tf.random_uniform`)

In [ ]:
n_vocab = len(int_to_vocab)
# print(n_vocab)
n_embedding = 200 # num of embedding features
with train_graph.as_default():
    embedding = tf.compat.v1.Variable(tf.compat.v1.random_uniform((n_vocab, n_embedding), -1, 1))
    embed = tf.compat.v1.nn.embedding_lookup(embedding, inputs)

### Negative Sampling

For every example we give the network we train it using output of softmax layer

For each input we make very small changes to millions of weights even though we have only one true example.

This makes training the network inefficient.

We can approximate the loss from softmax layer by only updating a small subset of all weigths at once.  $⇒$ update weights for correct label but only a small number of incorrect labels == Negative Sampling

`tf.nn.sampled_softmax_loss`

In [ ]:
# Create weights and biases for softmax layer
# Use tf.nn.sampled_softmax_loss to calculate the loss

n_sampled = 100
with train_graph.as_default():
    softmax_w = tf.compat.v1.Variable(tf.compat.v1.truncated_normal((n_vocab, n_embedding), stddev=0.1))
    softmax_b = tf.compat.v1.Variable(tf.compat.v1.zeros(n_vocab))
    # Calculate the loss using negative sampling
    loss = tf.compat.v1.nn.sampled_softmax_loss(softmax_w, softmax_b, labels, embed, n_sampled, n_vocab)
    cost = tf.compat.v1.reduce_mean(loss)
    optimizer = tf.compat.v1.train.AdamOptimizer().minimize(cost)

### Validation

Chose a few common words and few uncommon words. Then we will print out the closest words to them. It's a nice way to check that the embedding table is grouping together words with similar semantic meanings.

In [ ]:
with train_graph.as_default():
    # Random set of words to evaluate similarity on
    valid_size = 16
    valid_window = 100
    # Pick 8 samples from (0,100) and (1000, 1100) each ranges
    # Lower ids implies more frequent
    valid_examples = np.array(random.sample(range(valid_window), valid_size//2))
    valid_examples = np.append(valid_examples,
                               random.sample(range(1000, 1000 + valid_window), valid_size//2))

    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

    # We use the cosine distance:
    norm = tf.sqrt(tf.compat.v1.reduce_sum(tf.compat.v1.square(embedding), 1, keepdims=True))
    normalized_embedding = embedding / norm
    valid_embedding = tf.compat.v1.nn.embedding_lookup(normalized_embedding, valid_dataset)
    similarity = tf.compat.v1.matmul(valid_embedding, tf.compat.v1.transpose(normalized_embedding))


In [ ]:
epochs = 10
batch_size = 1000
window_size = 10

with train_graph.as_default():
    saver = tf.compat.v1.train.Saver()

with tf.compat.v1.Session(graph=train_graph) as sess:
    iteration = 1
    loss = 0
    sess.run(tf.compat.v1.global_variables_initializer())

    for e in tqdm(range(1, epochs+1)):
        batches = get_batches(train_words, batch_size, window_size)
        start = time.time()
        for x, y in batches:

            feed = {inputs: x, labels: np.array(y)[:, None]}
            train_loss, _ = sess.run([cost, optimizer], feed_dict=feed)

            loss += train_loss

            if iteration % 100 == 0:
                end = time.time()
                print("Epoch {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Avg. Training loss: {:.4f}".format(loss/100),
                      "{:.4f} sec/batch".format((end-start)/100))
                loss = 0
                start = time.time()

            if iteration % 1000 == 0:
                # note that this is expensive (~20% slowdown if computed every 500 steps)
                sim = similarity.eval()
                for i in range(valid_size):
                    valid_word = int_to_vocab[valid_examples[i]]
                    top_k = 8 # number of nearest neighbors
                    nearest = (-sim[i, :]).argsort()[1:top_k+1]
                    log = 'Nearest to %s:' % valid_word
                    for k in range(top_k):
                        close_word = int_to_vocab[nearest[k]]
                        log = '%s %s,' % (log, close_word)
                    print(log)

            iteration += 1
    save_path = saver.save(sess, "checkpoints/text8.ckpt")
    embed_mat = sess.run(normalized_embedding)

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1/10 Iteration: 100 Avg. Training loss: 5.6647 0.2929 sec/batch
Epoch 1/10 Iteration: 200 Avg. Training loss: 5.6407 0.2911 sec/batch
Epoch 1/10 Iteration: 300 Avg. Training loss: 5.5331 0.2920 sec/batch
Epoch 1/10 Iteration: 400 Avg. Training loss: 5.5721 0.2915 sec/batch
Epoch 1/10 Iteration: 500 Avg. Training loss: 5.5379 0.2907 sec/batch
Epoch 1/10 Iteration: 600 Avg. Training loss: 5.5234 0.2899 sec/batch
Epoch 1/10 Iteration: 700 Avg. Training loss: 5.5497 0.2934 sec/batch
Epoch 1/10 Iteration: 800 Avg. Training loss: 5.5489 0.2903 sec/batch
Epoch 1/10 Iteration: 900 Avg. Training loss: 5.4636 0.2910 sec/batch
Epoch 1/10 Iteration: 1000 Avg. Training loss: 5.4274 0.2915 sec/batch
Nearest to it: testable, bloc, kazakhstan, corrupting, chloroplasts, defibrillator, daito, adrenergic,
Nearest to can: private, specimens, blight, prevent, superpowered, bernese, surpassed, leah,
Nearest to also: wargame, grier, confessed, mesures, becket, sally, macgillivray, diffracted,
Nearest t

 10%|█         | 1/10 [22:28<3:22:17, 1348.65s/it]

Epoch 2/10 Iteration: 4700 Avg. Training loss: 4.6113 0.2109 sec/batch
Epoch 2/10 Iteration: 4800 Avg. Training loss: 4.5352 0.2937 sec/batch
Epoch 2/10 Iteration: 4900 Avg. Training loss: 4.5107 0.2922 sec/batch
Epoch 2/10 Iteration: 5000 Avg. Training loss: 4.5150 0.2914 sec/batch
Nearest to it: testable, truly, photosynthesis, intrigued, advising, daito, phaenomena, hermaphrodite,
Nearest to can: private, layer, superpowered, specimens, prevent, demystified, different, operas,
Nearest to also: strains, confessed, boron, becket, macgillivray, wargame, grier, appalachia,
Nearest to only: gangs, chessboard, clockwise, anglia, brightest, salvadoran, hyperspace, constants,
Nearest to years: seven, migration, gods, total, speeches, brazilian, wilmette, billy,
Nearest to he: draws, his, title, buckle, lay, lodges, belated, ecumenical,
Nearest to over: uncharacteristic, intend, today, angleton, mishna, joel, finkelstein, weakly,
Nearest to about: audubon, cpus, encrypt, lector, kowloon, fel

 20%|██        | 2/10 [44:59<3:00:01, 1350.13s/it]

...

Epoch 6/10 Iteration: 23200 Avg. Training loss: 4.0181 0.1743 sec/batch
Epoch 6/10 Iteration: 23300 Avg. Training loss: 3.9482 0.2918 sec/batch
Epoch 6/10 Iteration: 23400 Avg. Training loss: 3.9080 0.2904 sec/batch
Epoch 6/10 Iteration: 23500 Avg. Training loss: 3.9338 0.2918 sec/batch
Epoch 6/10 Iteration: 23600 Avg. Training loss: 3.9446 0.2911 sec/batch
Epoch 6/10 Iteration: 23700 Avg. Training loss: 3.9507 0.2908 sec/batch
Epoch 6/10 Iteration: 23800 Avg. Training loss: 3.9519 0.2917 sec/batch
Epoch 6/10 Iteration: 23900 Avg. Training loss: 3.9160 0.2912 sec/batch
Epoch 6/10 Iteration: 24000 Avg. Training loss: 3.9669 0.2922 sec/batch
Nearest to it: none, where, is, calculus, censorship, satisfiability, bloc, scoreboard,
Nearest to can: private, you, obstetrics, miracles, unable, operas, hazardous, layer,
Nearest to also: in, methodists, the, liechtenstein, youth, of, gohan, brief,
Nearest to only: hyperspace, have, each, chessboard, possible, they, pigeons, riverbed,
Nearest to y

 60%|██████    | 6/10 [2:14:56<1:29:57, 1349.39s/it]

Epoch 7/10 Iteration: 27800 Avg. Training loss: 3.9714 0.0938 sec/batch
Epoch 7/10 Iteration: 27900 Avg. Training loss: 3.9058 0.2911 sec/batch
Epoch 7/10 Iteration: 28000 Avg. Training loss: 3.9177 0.2913 sec/batch
Nearest to it: is, calculus, none, where, a, landmasses, calvinist, censorship,
Nearest to can: operas, primates, seventy, hazardous, private, manna, layer, niva,
Nearest to also: in, the, liechtenstein, of, and, as, methodists, strains,
Nearest to only: hyperspace, chessboard, pigeons, riverbed, they, have, each, possible,
Nearest to years: migration, seven, rate, thirty, total, fifteen, expectancy, over,
Nearest to he: his, graduating, him, draws, esteemed, returned, was, friend,
Nearest to over: airports, years, zero, dowland, ratio, female, oper, at,
Nearest to about: surveys, henslow, critique, kidby, than, historians, tycho, mabo,
Nearest to taking: consented, demosthenes, coleridge, variance, rescuing, angola, immunosuppressive, congestion,
Nearest to units: si, raci

 60%|██████    | 6/10 [2:33:19<1:42:13, 1533.27s/it]


KeyboardInterrupt: ignored

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

In [ ]:
viz_words = 500
tsne = TSNE()
embed_tsne = tsne.fit_transform(embed_mat[:viz_words, :])

In [ ]:
fig, ax = plt.subplots(figsize=(14, 14))
for idx in range(viz_words):
    plt.scatter(*embed_tsne[idx, :], color='steelblue')
    plt.annotate(int_to_vocab[idx], (embed_tsne[idx, 0], embed_tsne[idx, 1]), alpha=0.7)